# Mosaicking ASKAP Data

### This notebook serves to test the mosaicking algorithm code I have written for Apertif using Paolo Serra's IC 1459 ASKAP data.  He provided the covariance matrix, the beams shapes, and the individual and final data cubes.    

#### June 7, 2019 D.J. Pisano


For starters, we will use the Apercal environment to work with data.  

In [1]:
# Import needed packages
import os;
import shutil
import glob
import apercal.libs.lib as lib
import apercal
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from astropy.time import Time
from astropy.io import fits as fio
import subprocess

myusername = os.environ['USER']
if not ('PYTHONPATH' in os.environ and
        '/home/' + myusername + '/apercal' in os.environ['PYTHONPATH']):
      print("WARNING: your apercal directory should be in the $PYTHONPATH")

%config IPCompleter.greedy=True
%matplotlib notebook
lib.setup_logger('info', logfile='/home/{}/testing/logs/mosaic_test.log'.format(myusername))

cfgfile='/home/{}/testing/cfg/mosaic.cfg'.format(myusername)

basedir='/data4/pisano/ASKAP/'


%matplotlib inline



root - INFO : Logging started!
root - INFO : To see the log in a bash window use the following command:
root - INFO : tail -n +1 -f /home/pisano/testing/logs/mosaic_test.log


## We assume that we have a beam model/models and a correlation matrix already generated and saved.  

## We can now start importing data.

In [2]:
# Import images/beams into Miriad


def import_image(beam_num):
    # This function will import a FITS image into Miriad placing it in the mosaicdir
    fits = lib.miriad('fits')
    fits.op = 'xyin'
    fits.in_ = basedir+'nos_m{}_f3_R.fits'.format(str(beam_num))
    fits.out = basedir+'image_{}.map'.format(str(beam_num))
    fits.inp()
    fits.go()
    
def import_beam(beam_num):
    # This function will import the FITS image of a beam into Miriad format, placing it in the mosaicdir
    fits = lib.miriad('fits')
    fits.op = 'xyin'
    fits.in_ = basedir+'beam{}.fits'.format(str(beam_num))
    fits.out = basedir+'beam_{}o.map'.format(str(beam_num))
    fits.inp()
    fits.go()
        
def valid_beam(bm_num):
    # This function will determine if there is a valid image for a given beam
    if os.path.exists(basedir+'nos_m{}_f3_R.fits'.format(str(bm_num))):
        return True
    else:
        return False



In [3]:
# Import images from good beams 
beams=[]
for bm_num in range(9):
    if valid_beam(bm_num):
        beams.append(bm_num)
        
    
print(beams)

for beam in beams:
    import_image(beam)
    import_beam(beam)


miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/image_0.map, in_=/data4/pisano/ASKAP/nos_m0_f3_R.fits, op=xyin


[0, 1, 2, 3, 4, 5, 6, 7, 8]


miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/beam_0o.map, in_=/data4/pisano/ASKAP/beam0.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/image_1.map, in_=/data4/pisano/ASKAP/nos_m1_f3_R.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/beam_1o.map, in_=/data4/pisano/ASKAP/beam1.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/image_2.map, in_=/data4/pisano/ASKAP/nos_m2_f3_R.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/beam_2o.map, in_=/data4/pisano/ASKAP/beam2.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/image_3.map, in_=/data4/pisano/ASKAP/nos_m3_f3_R.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/beam_3o.map, in_=/data4/pisano/ASKAP/beam3.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/image_4.map, in_=/data4/pisano/ASKAP/nos_m4_f3_R.fits, op=xyin
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/beam_4o.map, in_

In [7]:
# Cut beams below 25% of peak

maths = lib.miriad('maths')
for beam in beams:
    maths.exp="'<"+basedir+"beam_{}o.map>'".format(str(beam))
    maths.mask="'<"+basedir+"beam_{}o.map>.gt.0.25'".format(str(beam))
    maths.out=basedir+"beam_{}b.map".format(str(beam))
    maths.inp()
    maths.go()
    
imblr = lib.miriad('imblr')
for beam in beams:
    imblr.in_=basedir+'beam_{}b.map'.format(str(beam))
    imblr.out=basedir+'beam_{}.map'.format(str(beam))
    imblr.go()



miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/beam_0o.map>.gt.0.25', exp='</data4/pisano/ASKAP/beam_0o.map>', out=/data4/pisano/ASKAP/beam_0b.map
miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/beam_1o.map>.gt.0.25', exp='</data4/pisano/ASKAP/beam_1o.map>', out=/data4/pisano/ASKAP/beam_1b.map
miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/beam_2o.map>.gt.0.25', exp='</data4/pisano/ASKAP/beam_2o.map>', out=/data4/pisano/ASKAP/beam_2b.map
miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/beam_3o.map>.gt.0.25', exp='</data4/pisano/ASKAP/beam_3o.map>', out=/data4/pisano/ASKAP/beam_3b.map
miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/beam_4o.map>.gt.0.25', exp='</data4/pisano/ASKAP/beam_4o.map>', out=/data4/pisano/ASKAP/beam_4b.map
miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/beam_5o.map>.gt.0.25', exp='</data4/pisano/ASKAP/beam_5o.map>', out=/data4/pisano/ASKAP/beam_5b.map
miriad maths - INFO : task=maths, 

In [8]:
# Read in noise covariance matrix from 'covariance.fits' file.   
cov_file=fio.open(basedir+'covariance.fits')

#cov_file.info()

noise_cov=cov_file[0].data
cov_file.close()


# Only the inverse of this matrix is ever used:
inv_cov=np.linalg.inv(noise_cov)

print(inv_cov)
print(np.min(inv_cov))
print(np.max(inv_cov))

print(np.linalg.inv(inv_cov)-noise_cov)

[[ 1.41610527e+04 -1.38920404e+03 -1.80324965e+03 -2.30224814e+03
  -1.41717670e+03 -2.22651525e+03 -2.01086911e+03  5.92553569e+02
   5.58226698e+02]
 [-1.38920404e+03  1.30057886e+04 -2.32714340e+03  3.99722675e+02
  -6.78011245e+01  4.48894881e+02 -2.20192329e+03  2.56280058e+02
  -7.07989879e+00]
 [-1.80324965e+03 -2.32714340e+03  1.40892295e+04 -1.58012499e+03
   3.90634851e+02 -1.23999059e+02  7.96371183e+02 -2.20911968e+03
  -2.69356314e+02]
 [-2.30224814e+03  3.99722675e+02 -1.58012499e+03  1.41952390e+04
  -2.64115223e+03  9.72066371e+02 -7.46447386e+01 -1.46014628e+03
  -2.97269962e+02]
 [-1.41717670e+03 -6.78011245e+01  3.90634851e+02 -2.64115223e+03
   1.38187928e+04 -2.84389418e+03  4.98317590e+02 -2.29627349e+01
  -4.24010627e+01]
 [-2.22651525e+03  4.48894881e+02 -1.23999059e+02  9.72066371e+02
  -2.84389418e+03  1.50285071e+04 -1.89912660e+03 -1.93536336e+02
  -1.58589336e+03]
 [-2.01086911e+03 -2.20192329e+03  7.96371183e+02 -7.46447386e+01
   4.98317590e+02 -1.8991266

In [9]:
# Get info in order to convolve images to same synthesized beam size

# Extract beam parameters from headers
bmaj=[]
bmin=[]
bpa=[]
for beam in beams:
    gethd = lib.miriad('gethd')
    gethd.in_ = basedir+'image_{}.map/bmaj'.format(str(beam))
    bmaj.append(gethd.go())
    gethd.in_ = basedir+'image_{}.map/bmin'.format(str(beam))
    bmin.append(gethd.go())
    gethd.in_ = basedir+'image_{}.map/bpa'.format(str(beam))
    bpa.append(gethd.go())
    
# Calculate maximum bmaj and bmin and median bpa for final convolved beam shape
bmajor = [float(x[0]) for x in bmaj]
bmajor = 3600.*np.degrees(bmajor)

bminor = [float(x[0]) for x in bmin]
bminor = 3600.*np.degrees(bminor)

bangle = [float(x[0]) for x in bpa]
#bangle = np.degrees(bangle)

c_beam = [1.1*np.nanmax(bmajor),1.1*np.nanmax(bminor),np.nanmedian(bangle)]
print('The final, convolved, synthesized beam has bmaj, bmin, bpa of: ',c_beam)

('The final, convolved, synthesized beam has bmaj, bmin, bpa of: ', [78.5677106158108, 61.03863433118356, -76.54572])


In [10]:
# Convolve all images to same synthesized beam (c_beam)

for beam in beams:
    convol=lib.miriad('convol')
    convol.map = basedir+'image_{}.map'.format(str(beam))
    convol.out = basedir+'image_{}_convol.map'.format(str(beam))
    convol.fwhm = '{0},{1}'.format(str(c_beam[0]),str(c_beam[1]))
    convol.pa = c_beam[2]
    convol.options = 'final'
    convol.inp()
    convol.go()

miriad convol - INFO : map=/data4/pisano/ASKAP/image_0.map, task=convol, pa=-76.54572, fwhm=78.5677106158108,61.03863433118356, options=final, out=/data4/pisano/ASKAP/image_0_convol.map
miriad convol - INFO : map=/data4/pisano/ASKAP/image_1.map, task=convol, pa=-76.54572, fwhm=78.5677106158108,61.03863433118356, options=final, out=/data4/pisano/ASKAP/image_1_convol.map
miriad convol - INFO : map=/data4/pisano/ASKAP/image_2.map, task=convol, pa=-76.54572, fwhm=78.5677106158108,61.03863433118356, options=final, out=/data4/pisano/ASKAP/image_2_convol.map
miriad convol - INFO : map=/data4/pisano/ASKAP/image_3.map, task=convol, pa=-76.54572, fwhm=78.5677106158108,61.03863433118356, options=final, out=/data4/pisano/ASKAP/image_3_convol.map
miriad convol - INFO : map=/data4/pisano/ASKAP/image_4.map, task=convol, pa=-76.54572, fwhm=78.5677106158108,61.03863433118356, options=final, out=/data4/pisano/ASKAP/image_4_convol.map
miriad convol - INFO : map=/data4/pisano/ASKAP/image_5.map, task=convo

In [11]:
# Now, we need to do the matrix math

# First calculate transpose of beam matrix multiplied by the inverse covariance matrix
# Will use *maths* and *imcomb* in Miriad

# Using "beams" list to account for missing beams/images
# Only doing math where inv_cov value is non-zero

maths = lib.miriad('maths')
for bm in beams:
    for b in beams:
        maths.out = basedir+'tmp_{}.map'.format(str(b))
        if inv_cov[b,bm]!=0.:
                operate="'<"+basedir+"beam_{0}.map>*({1})'".format(str(b),inv_cov[b,bm])
        maths.exp = operate
        maths.options='unmask'
        maths.inp()
        maths.go()
    i=1
    while i<len(beams):
        if i==1:
            operate = "'<"+basedir+"tmp_{}.map>+<".format(str(beams[i-1]))+basedir+"tmp_{}.map>'".format(str(beams[i]))
        else:
            operate="'<"+basedir+"tmp_{}.map>".format(str(beams[i]))+"+<"+basedir+"sum_{}.map>'".format(str(beams[i-1]))
        maths.out = basedir+'sum_{}.map'.format(str(beams[i]))
        maths.exp = operate
        maths.options='unmask'
        maths.inp()
        maths.go()
        i+=1
        
    os.rename(basedir+'sum_{}.map'.format(str(beams[i-1])),basedir+'btci_{}.map'.format(str(bm)))

    for fl in glob.glob(basedir+'tmp_*.map'):
        shutil.rmtree(fl)
    for fl in glob.glob(basedir+'sum_*.map'):
        shutil.rmtree(fl)


miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_0.map>*(14161.0526932)', out=/data4/pisano/ASKAP/tmp_0.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_1.map>*(-1389.2040353)', out=/data4/pisano/ASKAP/tmp_1.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_2.map>*(-1803.24964619)', out=/data4/pisano/ASKAP/tmp_2.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_3.map>*(-2302.24814337)', out=/data4/pisano/ASKAP/tmp_3.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_4.map>*(-1417.17670022)', out=/data4/pisano/ASKAP/tmp_4.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_5.map>*(-2226.51524708)', out=/data4/pisano/ASKAP/tmp_5.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_6.map>*(-2010.86911195)', out=/data4/pisano/ASKAP/tmp_6.map
miriad maths - 

miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_5.map>*(972.066370658)', out=/data4/pisano/ASKAP/tmp_5.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_6.map>*(-74.6447385823)', out=/data4/pisano/ASKAP/tmp_6.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_7.map>*(-1460.14628192)', out=/data4/pisano/ASKAP/tmp_7.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/beam_8.map>*(-297.26996248)', out=/data4/pisano/ASKAP/tmp_8.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_0.map>+</data4/pisano/ASKAP/tmp_1.map>', out=/data4/pisano/ASKAP/sum_1.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_2.map>+</data4/pisano/ASKAP/sum_1.map>', out=/data4/pisano/ASKAP/sum_2.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_3.map>+</data4/pisano/ASKAP/sum_2.map>', out=/da

miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_2.map>+</data4/pisano/ASKAP/sum_1.map>', out=/data4/pisano/ASKAP/sum_2.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_3.map>+</data4/pisano/ASKAP/sum_2.map>', out=/data4/pisano/ASKAP/sum_3.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_4.map>+</data4/pisano/ASKAP/sum_3.map>', out=/data4/pisano/ASKAP/sum_4.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_5.map>+</data4/pisano/ASKAP/sum_4.map>', out=/data4/pisano/ASKAP/sum_5.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_6.map>+</data4/pisano/ASKAP/sum_5.map>', out=/data4/pisano/ASKAP/sum_6.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/tmp_7.map>+</data4/pisano/ASKAP/sum_6.map>', out=/data4/pisano/ASKAP/sum_7.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/

In [12]:
# Calculate variance map (using beams and noise covariance matrix over entire map)
# This is the denominator for I(mosaic)

maths = lib.miriad('maths')
i=0
for bm in beams:
    operate="'<"+basedir+"btci_{}.map>*<".format(str(bm))+basedir+"beam_{}.map>'".format(str(bm))
    if bm!=beams[0]:
        operate=operate[:-1]+"+<"+basedir+"out_{}_mos.map>'".format(str(i))
    i+=1
    maths.out = basedir+"out_{}_mos.map".format(str(i))
    maths.exp = operate
    maths.options='unmask'
    maths.inp()
    maths.go()

os.rename(basedir+'out_{}_mos.map'.format(str(i)),basedir+'variance_mos.map')
   

miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/btci_0.map>*</data4/pisano/ASKAP/beam_0.map>', out=/data4/pisano/ASKAP/out_1_mos.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/btci_1.map>*</data4/pisano/ASKAP/beam_1.map>+</data4/pisano/ASKAP/out_1_mos.map>', out=/data4/pisano/ASKAP/out_2_mos.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/btci_2.map>*</data4/pisano/ASKAP/beam_2.map>+</data4/pisano/ASKAP/out_2_mos.map>', out=/data4/pisano/ASKAP/out_3_mos.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/btci_3.map>*</data4/pisano/ASKAP/beam_3.map>+</data4/pisano/ASKAP/out_3_mos.map>', out=/data4/pisano/ASKAP/out_4_mos.map
miriad maths - INFO : task=maths, options=unmask, exp='</data4/pisano/ASKAP/btci_4.map>*</data4/pisano/ASKAP/beam_4.map>+</data4/pisano/ASKAP/out_4_mos.map>', out=/data4/pisano/ASKAP/out_5_mos.map
miriad maths - INFO : task=maths, options=unmask, e

In [13]:
# Calculate transpose of beam matrix multiplied by noise_cov multiplied by image from each beam for each position
# in the final image

maths = lib.miriad('maths')
i=0
for bm in beams:
    #operate="'<"+basedir+"image_{}_convol.map>*<".format(str(bm))+basedir+"btci_{}.map>'".format(str(bm))
    operate="'<"+basedir+"image_{}.map>*<".format(str(bm))+basedir+"btci_{}.map>'".format(str(bm))
    if bm!=beams[0]:
        operate=operate[:-1]+"+<"+basedir+"mos_{}.map>'".format(str(i))
    i+=1
    maths.out = basedir+"mos_{}.map".format(str(i))
    maths.exp = operate
    maths.options='unmask,grow'
    maths.inp()
    maths.go()
os.rename(basedir+'mos_{}.map'.format(str(i)),basedir+'mosaic_im.map')


miriad maths - INFO : task=maths, options=unmask,grow, exp='</data4/pisano/ASKAP/image_0.map>*</data4/pisano/ASKAP/btci_0.map>', out=/data4/pisano/ASKAP/mos_1.map
miriad maths - INFO : task=maths, options=unmask,grow, exp='</data4/pisano/ASKAP/image_1.map>*</data4/pisano/ASKAP/btci_1.map>+</data4/pisano/ASKAP/mos_1.map>', out=/data4/pisano/ASKAP/mos_2.map
miriad maths - INFO : task=maths, options=unmask,grow, exp='</data4/pisano/ASKAP/image_2.map>*</data4/pisano/ASKAP/btci_2.map>+</data4/pisano/ASKAP/mos_2.map>', out=/data4/pisano/ASKAP/mos_3.map
miriad maths - INFO : task=maths, options=unmask,grow, exp='</data4/pisano/ASKAP/image_3.map>*</data4/pisano/ASKAP/btci_3.map>+</data4/pisano/ASKAP/mos_3.map>', out=/data4/pisano/ASKAP/mos_4.map
miriad maths - INFO : task=maths, options=unmask,grow, exp='</data4/pisano/ASKAP/image_4.map>*</data4/pisano/ASKAP/btci_4.map>+</data4/pisano/ASKAP/mos_4.map>', out=/data4/pisano/ASKAP/mos_5.map
miriad maths - INFO : task=maths, options=unmask,grow, ex

In [14]:
# Find maximum value of variance map
imstat = lib.miriad('imstat')
imstat.in_="'"+basedir+"variance_mos.map'"
imstat.region="'quarter(1)'"
imstat.axes="'x,y'"
a=imstat.go()

# Always outputs max value at same point
var_max=a[-1].split(" ")[-4]
print(var_max)

1.509E+04


In [15]:
# Divide image by variance map

maths = lib.miriad('maths')
maths.out = basedir+'mosaic_final.map'
maths.exp="'<"+basedir+"mosaic_im.map>/<"+basedir+"variance_mos.map>'"
maths.mask="'<"+basedir+"variance_mos.map>.gt.0.0625*"+str(var_max)+"'"
maths.options='grow'
maths.inp()
maths.go()


miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/variance_mos.map>.gt.0.0625*1.509E+04', options=grow, exp='</data4/pisano/ASKAP/mosaic_im.map>/</data4/pisano/ASKAP/variance_mos.map>', out=/data4/pisano/ASKAP/mosaic_final.map


['', 'maths: Revision 1.7, 2018/11/29 23:34:06 UTC', '']

In [16]:
# Produce mosaic noise map
maths = lib.miriad('maths')
maths.out = basedir+'mosaic_noise.map'
maths.exp="'1./sqrt(<"+basedir+"variance_mos.map>)'"
maths.mask="'<"+basedir+"variance_mos.map>.gt.0.0625*"+str(var_max)+"'"
maths.inp()
maths.go()

puthd = lib.miriad('puthd')
puthd.in_=basedir+'mosaic_noise.map/bunit'
puthd.value='JY/BEAM'
puthd.go()

miriad maths - INFO : task=maths, mask='</data4/pisano/ASKAP/variance_mos.map>.gt.0.0625*1.509E+04', exp='1./sqrt(</data4/pisano/ASKAP/variance_mos.map>)', out=/data4/pisano/ASKAP/mosaic_noise.map


['', 'puthd: Revision 1.3, 2011/09/15 05:53:36 UTC', '']

In [17]:
# Clean up files
for fl in glob.glob(basedir+'*_convol.map'):
    shutil.rmtree(fl)

shutil.rmtree(basedir+'mosaic_im.map')

for fl in glob.glob(basedir+'mos_*.map'):
    shutil.rmtree(fl)

for fl in glob.glob(basedir+'btci_*.map'):
    shutil.rmtree(fl)
    
for fl in glob.glob(basedir+'out_*_mos.map'):
    shutil.rmtree(fl)

In [20]:
# Write out FITS files
fits = lib.miriad('fits')
fits.op='xyout'
fits.in_=basedir+'mosaic_final.map'
fits.out=basedir+'mosaic_final.fits'
fits.inp()
fits.go()

fits.in_=basedir+'mosaic_noise.map'
fits.out=basedir+'mosaic_noise.fits'
fits.inp()
fits.go()

miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/mosaic_final.fits, in_=/data4/pisano/ASKAP/mosaic_final.map, op=xyout
miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/mosaic_noise.fits, in_=/data4/pisano/ASKAP/mosaic_noise.map, op=xyout


['', 'fits: Revision 1.33, 2018/12/04 04:02:11 UTC', '']

In [18]:
# Import original cube
fits = lib.miriad('fits')
fits.op='xyin'
fits.in_=basedir+'moscov.fits'
fits.out=basedir+'mosaic_orig.map'
fits.inp()
fits.go()


miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/mosaic_orig.map, in_=/data4/pisano/ASKAP/moscov.fits, op=xyin


['',
 'fits: Revision 1.33, 2018/12/04 04:02:11 UTC',
 '',
 'Phase and pointing centres differ ... saving pointing information',
 'There were no blanked pixels in the input']

In [19]:
# Difference mosaic cubes

maths = lib.miriad('maths')
maths.exp = "'<"+basedir+"mosaic_final.map>-<"+basedir+"mosaic_orig.map>'"
maths.out = basedir+'mosaic_diff.map'
maths.go()

['', 'maths: Revision 1.7, 2018/11/29 23:34:06 UTC', '']

In [21]:
# Write out difference cube to fits
fits = lib.miriad('fits')
fits.op='xyout'
fits.in_=basedir+'mosaic_diff.map'
fits.out=basedir+'mosaic_diff.fits'
fits.inp()
fits.go()


miriad fits - INFO : task=fits, out=/data4/pisano/ASKAP/mosaic_diff.fits, in_=/data4/pisano/ASKAP/mosaic_diff.map, op=xyout


['', 'fits: Revision 1.33, 2018/12/04 04:02:11 UTC', '']

In [ ]:
# Create difference map of mosaic noise
fits = lib.miriad('fits')
fits.op='xyin'
fits.in_=basedir+'rmscov.fits'
fits.out=basedir+'rms_orig.map'
fits.inp()
fits.go()

# Difference noise maps

maths = lib.miriad('maths')
maths.exp = "'<"+basedir+"mosaic_noise.map>-<"+basedir+"rms_orig.map>'"
maths.out = basedir+'rms_diff.map'
maths.inp()
maths.go()

# Write out difference map to fits
fits = lib.miriad('fits')
fits.op='xyout'
fits.in_=basedir+'rms_diff.map'
fits.out=basedir+'rms_diff.fits'
fits.inp()
fits.go()
